In [1]:
# Attempt to rollup most of NSE programs into ONE

# STATUS: Incomplete (Trial with index options)
# Run-time: Unknown

# Dependencies:
#  - List of indexes and equities
#  - underlying's ohlc history for standard deviation and Black Scholes

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 7496, clientId=5) # kavi tws live
# ib = IB().connect('127.0.0.1', 4001, clientId=5) # kavi IBG live
# ib = IB().connect('127.0.0.1', 7499, clientId=5) # kavi tws paper-trade
# ib = IB().connect('127.0.0.1', 4004, clientId=5) # kavi IBG paper-trade

In [ ]:
#***      Imports    *****
import pandas as pd
from itertools import product

#***    Paths and Variables    ***
datapath = r'./zdata/'
market = 'NSE'

#***    Get scrips from IBKR   ***
df_ib = pd.DataFrame([]) # Initialize

# from IBKR's symbols page
url = "https://www.interactivebrokers.com.hk/en/index.php?"
url = url+"f=2222&exch=nse&showcategories=OPTGRP&page="

for i in range(4):
    df_ib = df_ib.append(df_ib.append(pd.read_html(url+str(i), header=0)[2]))

# Exit program if dataframe is empty
if df_ib.empty:
    print('Interactive Brokers dataframe is empty!!!')
    sys.exit(0)
    
df_ib = df_ib.drop_duplicates().reset_index(drop=True)

# Qualify Stock contracts
stocks = [Stock(symbol=s, exchange=market) for s in df_ib['IB Symbol']]
nse_qs = ib.qualifyContracts(*stocks)

df_qs = util.df(nse_qs) # data frame of qualified stocks

# Get a list of indexes
df_ix = df_ib[~df_ib['IB Symbol'].isin(df_qs.symbol)]

# Qualify the indexes
indexes = [Index(symbol=s, exchange='NSE') for s in df_ix['IB Symbol']]
nse_qi = ib.qualifyContracts(*indexes)

df_qi = util.df(nse_qi) # dataframe of qualified indexes

In [33]:
symbols = nse_qi + nse_qs

symbols = nse_qi # DATA LIMITER!!!

#*** get the option chains  ***
chains = {s: ib.reqSecDefOptParams(s.symbol, '', s.secType, s.conId) for s in symbols}
# chains = [ib.reqSecDefOptParams(s.symbol, '', s.secType, s.conId) for s in symbols]
chains

df_und = util.df(chains) # dataframe of underlying chains

In [39]:
tup_chains = [(k.symbol, tuple(i.expirations), tuple(i.strikes)) 
 for k, v in chains.items() 
 for i in v]

In [46]:
tup_chains

[('BANKNIFTY',
  ('20190228',
   '20190110',
   '20190124',
   '20181213',
   '20181227',
   '20190103',
   '20190117',
   '20181220',
   '20190207',
   '20190131'),
  (25600.0,
   23300.0,
   26500.0,
   24200.0,
   27400.0,
   25100.0,
   28300.0,
   22800.0,
   26000.0,
   23700.0,
   26900.0,
   24600.0,
   27800.0,
   25500.0,
   23200.0,
   26400.0,
   24100.0,
   27000.0,
   27300.0,
   25000.0,
   28200.0,
   25900.0,
   23600.0,
   26800.0,
   24500.0,
   27700.0,
   25400.0,
   23100.0,
   26300.0,
   24000.0,
   27200.0,
   24900.0,
   28100.0,
   25800.0,
   23500.0,
   26700.0,
   24400.0,
   27600.0,
   25300.0,
   27900.0,
   28500.0,
   23000.0,
   26200.0,
   23900.0,
   27100.0,
   24800.0,
   28000.0,
   25700.0,
   23400.0,
   26600.0,
   24300.0,
   27500.0,
   25200.0,
   28400.0,
   22900.0,
   26100.0,
   23800.0,
   24700.0)),
 ('NIFTY50',
  ('20190228',
   '20190627',
   '20200625',
   '20181227',
   '20220630',
   '20211230',
   '20230629',
   '20190926',
   

In [44]:
symbols = [i[0] for i in tup_chains]
expiries = [i[1] for i in tup_chains]
strikes = [i[2] for i in tup_chains]
rights = ['P', 'C']

[Option(symbol=s,  lastTradeDateOrContractMonth=e, strike=k, right=r, exchange='NSE')
for s in symbols
for e in expiries
for k in strikes
for r in rights]


[Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth=('20190228', '20190110', '20190124', '20181213', '20181227', '20190103', '20190117', '20181220', '20190207', '20190131'), strike=(25600.0, 23300.0, 26500.0, 24200.0, 27400.0, 25100.0, 28300.0, 22800.0, 26000.0, 23700.0, 26900.0, 24600.0, 27800.0, 25500.0, 23200.0, 26400.0, 24100.0, 27000.0, 27300.0, 25000.0, 28200.0, 25900.0, 23600.0, 26800.0, 24500.0, 27700.0, 25400.0, 23100.0, 26300.0, 24000.0, 27200.0, 24900.0, 28100.0, 25800.0, 23500.0, 26700.0, 24400.0, 27600.0, 25300.0, 27900.0, 28500.0, 23000.0, 26200.0, 23900.0, 27100.0, 24800.0, 28000.0, 25700.0, 23400.0, 26600.0, 24300.0, 27500.0, 25200.0, 28400.0, 22900.0, 26100.0, 23800.0, 24700.0), right='P', exchange='NSE'),
 Option(symbol='BANKNIFTY', lastTradeDateOrContractMonth=('20190228', '20190110', '20190124', '20181213', '20181227', '20190103', '20190117', '20181220', '20190207', '20190131'), strike=(25600.0, 23300.0, 26500.0, 24200.0, 27400.0, 25100.0, 28300.0, 22800.0, 260

In [ ]:
#...... qualify options and strikes

rights=['P'] # currently taking puts only

L = [[[x[0]], [x[1]], x[2], x[3], rights] for x in tup_chains]
df1 = pd.DataFrame([j for i in L for j in product(*i)], 
             columns=['ibSymbol', 'undConId', 'expiry', 'strike', 'right'])

In [ ]:
df = df1[:3] # !!!!DATA LIMITER!!!

options = [Option(symbol=s,  lastTradeDateOrContractMonth=e, strike=k, right=r, exchange='NSE')
for s in df.ibSymbol
for e in df.expiry
for k in df.strike
for r in df.right]

qual_options = ib.qualifyContracts(*options)

In [ ]:
df

In [ ]:
qual_options

# To-do
* something is wrong in the qual_options. Fix it.
* integration with standard deviation
* remove unnecessary options (Put)
* integration with black scholes
* integration with lot and margin
* generate orders
* place orders (what-if) to check margins (duplicate)
* check latest prices
* screen most valueable RoM and PoP
* place orders